In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [8]:
cd 'drive/My Drive/Colab Notebooks/DataWorkshopMatrix/matrix_one'

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/DataWorkshopMatrix/matrix_one'
/content/drive/My Drive/Colab Notebooks/DataWorkshopMatrix/matrix_one


In [9]:
ls

data/       day_3_to_csv.ipynb  day4.ipynb
day3.ipynb  day_4_done.PNG      Untitled0.ipynb


In [10]:
df = pd.read_csv('data/men_shoes.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
def run_model(feats,model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [17]:
df['brand_cat']=df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [18]:
model = RandomForestRegressor(max_depth=5,n_estimators=100,random_state=101)
run_model(['brand_cat'],model)

(-57.26733564291733, 4.228035385534019)

In [22]:
df.features[0]

'[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

In [21]:
test = {'key':'value'}

test['key']

str(test)

"{'key': 'value'}"

In [23]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [0]:
def parse_features(x):
  output_dict ={}
  if str(x) == 'nan': return output_dict
  
  features =  literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    
    output_dict[key] = value

  return output_dict
df['features_parsed'] = df['features'].map(parse_features)

In [42]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [44]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [45]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_is wheeled', 'feat_features', 'feat_closure style',
       'feat_lens material:', 'feat_vendor description', 'feat_capacity',
       'feat_style #', 'feat_disclosures', 'feat_item character',
       'feat_boot height'],
      dtype='object', length=526)

In [46]:
df[ df['feat_athlete'].isnull()].shape

(18272, 526)

In [48]:
df.shape[0]

18280

In [53]:
df[ False == df['feat_athlete'].isnull()].shape[0] / df.shape[0] * 100

0.0437636761487965

In [0]:
key_stats={}
for key in keys:
  key_stats[key]=df[ False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [60]:
{k:v for k,v in key_stats.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat']=df['feat_brand'].factorize()[0]
df['feat_color_cat']=df['feat_color'].factorize()[0]
df['feat_gender_cat']=df['feat_gender'].factorize()[0]
df['feat_material_cat']=df['feat_material'].factorize()[0]
df['feat_manufacturer part number_cat']=df['feat_manufacturer part number'].factorize()[0]

df['feat_sport_cat']=df['feat_sport'].factorize()[0]
df['feat_style_cat']=df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
  

In [0]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())

In [66]:
df[df.brand == df.feat_brand].shape

(8846, 531)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]
#feats_cat

In [0]:
model = RandomForestRegressor(max_depth=5,n_estimators=100,random_state=101)
feats = ['brand_cat','feat_brand_cat','feat_gender_cat','feat_material_cat','feat_style_cat','feat_sport_cat']
feats +=  ['feat_hardsided or softsided_cat',
 'feat_number of batteries included_cat',
 'feat_dimensions_cat',
 'feat_transactionid_cat',
 'feat_date_cat',
 'feat_foot arch_cat',
 'feat_color family_cat',
 'feat_case type_cat',
 'feat_age end_cat',
 'feat_best sellers rank_cat',
 'feat_upper material_cat',
 'feat_item weight unit of measure_cat',
 'feat_occasion_cat',
 'feat_country of manufacturer_cat',
 'feat_lens type_cat',
 'feat_walmart no._cat',
 'feat_jacket length_cat',
 'feat_materials_cat',
 'feat_vehicle type_cat',
 'feat_item package quantity_cat',
 'feat_global composite sports type_cat',
 'feat_protects against_cat',
 'feat_has paper wood_cat',
 'feat_leather :_cat',
 'feat_number of pieces_cat',
 'feat_includes_cat',
 'feat_series name_cat',
 'feat_sizearm_cat',
 'feat_is wheeled_cat',
 'feat_features_cat',
 'feat_closure style_cat',
 'feat_lens material:_cat',
 'feat_vendor description_cat',
 'feat_capacity_cat',
 'feat_style #_cat',
 'feat_disclosures_cat',
 'feat_item character_cat',
 'feat_boot height_cat']
feats = list(set(feats))
results = run_model(feats,model)

In [97]:
X = df[feats].values
y = df['prices_amountmin'].values


m = RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
m.fit(X,y)
print(results)
perm = PermutationImportance(m,random_state=1).fit(X,y);
eli5.show_weights(perm,feature_names=feats)

(-57.38316266260963, 4.483226126392516)


Weight,Feature
0.2578 ± 0.0130,brand_cat
0.0914 ± 0.0114,feat_material_cat
0.0398 ± 0.0021,feat_gender_cat
0.0184 ± 0.0013,feat_brand_cat
0.0043 ± 0.0005,feat_jacket length_cat
0.0041 ± 0.0004,feat_style_cat
0.0039 ± 0.0006,feat_is wheeled_cat
0.0028 ± 0.0001,feat_item package quantity_cat
0.0027 ± 0.0002,feat_lens material:_cat
0.0012 ± 0.0002,feat_occasion_cat


In [87]:
df['brand'].value_counts(normalize=True)

nike                           0.097210
puma                           0.033315
ralph lauren                   0.028775
vans                           0.021116
new balance                    0.020295
                                 ...   
character                      0.000055
new era                        0.000055
pentair                        0.000055
adidas crazy1 kobe bb shoes    0.000055
calcutta                       0.000055
Name: brand, Length: 1732, dtype: float64

In [88]:
df [df.brand == 'nike'].features_parsed.sample(5).values

array([{'style': 'athletic sneakers', 'material': 'leather', 'condition': 'new with box'},
       {'size': '9.5', 'clothing size': 'kids 9 plus', 'brand': 'nike', 'color': 'green'},
       {'main color': 'black/orange/white', 'vendor description': 'shox fuse metal cleat', 'material': 'mesh', 'condition': 'new with box'},
       {'gender': 'girls', 'shoe category': 'girls shoes', 'color': 'white/vivid pink/pure platinum', 'casual & dress shoe style': 'athletic', 'manufacturer part number': '644446166', 'brand': 'nike', 'shoe width': 'medium'},
       {'style': 'running, cross training', 'condition': 'new with box'}],
      dtype=object)

In [0]:
!git add day5.ipynb

In [0]:
  !git config --global user.email "nickypolit2@gmail.com"
  !git config --global user.name "Nikita"